In [ ]:
from utils.visualization_utils import get_att_map, objdict, get_dict
from scipy.stats import multivariate_normal
import keras.backend as K

import numpy as np
import os
from PIL import Image
import json
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Let's create initial attention.
im_width = 14
im_height = 14

def create_gaussian(center):
    xlim = (-3, 3)
    ylim = (-3, 3)
    kernel = multivariate_normal(mean=center, cov=np.eye(2))
    x = np.linspace(xlim[0], xlim[1], im_width)
    y = np.linspace(ylim[0], ylim[1], im_height)
    xx, yy = np.meshgrid(x,y)
    xxyy = np.c_[xx.ravel(), yy.ravel()]
    zz = kernel.pdf(xxyy)
    in_att = zz.reshape((im_height, im_width))
    return in_att

in_att = create_gaussian((0, 0))
plt.imshow(in_att)
plt.show()

In [ ]:
###################
data_type = "vrd"
###################
if data_type=="vrd":
    nrows=7
    ncols=5
    ssn_checkpoint = "/data/ranjaykrishna/ReferringRelationships/temp/vrd_ssn_convs3/model29-1.33.h5"
    sym_ssn_checkpoint = "/data/ranjaykrishna/ReferringRelationships/temp/vrd_sym_ssn_convs3/model29-1.29.h5"
    vocab_dir = os.path.join('data/VRD')
elif data_type=="clevr":
    nrows=3
    ncols=2
    ssn_checkpoint = "/data/ranjaykrishna/ReferringRelationships/temp/clevr_ssn/model03-0.15.h5"
    sym_ssn_checkpoint = "/data/ranjaykrishna/ReferringRelationships/temp/clevr_sym_ssn_convs3_iterations2/model00-0.18.h5"
    #annotations_test = json.load(open("/data/chami/ReferringRelationships/data/Clevr/annotations_test.json"))
    #img_dir = '/data/chami/ReferringRelationships/data/Clevr/images/val'
    vocab_dir = os.path.join('/data/chami/ReferringRelationships/data/Clevr/')
predicate_dict, obj_subj_dict = get_dict(vocab_dir)

In [ ]:
# Grab all the weights
model_weights = h5py.File(sym_ssn_checkpoint)
params = objdict(json.load(open(os.path.join(os.path.dirname(sym_ssn_checkpoint), "args.json"), "r")))
conv_filters = {}
inv_conv_filters = {}
for i in range(params.num_predicates):
    predicate = predicate_dict[i]
    conv_filters[predicate] = []
    inv_conv_filters[predicate] = []
    for j in range(params.nb_conv_att_map):
        if 'conv0-predicate0' in model_weights:
            conv_weights_name = "conv{}-predicate{}".format(j, i)
            inv_conv_weights_name = "conv{}-inv-predicate{}".format(j, i)
        else:
            conv_weights_name = "conv{}-predicate{}-0".format(j, i)
            inv_conv_weights_name = "conv{}-predicate{}-1".format(j, i)
        if j == 0:
            conv_filters[predicate] += [model_weights[conv_weights_name][conv_weights_name]['kernel:0'][()]]
            inv_conv_filters[predicate] += [model_weights[inv_conv_weights_name][inv_conv_weights_name]['kernel:0'][()]]
        else:
            conv_filters[predicate] += [model_weights[conv_weights_name][conv_weights_name]['kernel:0'][()]]
            inv_conv_filters[predicate] += [model_weights[inv_conv_weights_name][inv_conv_weights_name]['kernel:0'][()]]

In [ ]:
###################
predicate = "inside"
###################
sess = tf.InteractiveSession()
att = in_att.reshape(1, im_height, im_width, 1)
att = K.constant(att)
for j in range(params.nb_conv_att_map):
    kernel = np.array(conv_filters[predicate][j])
    att = K.conv2d(att, kernel, padding='same', data_format='channels_last')
    att = K.relu(att)
att = K.sum(att, axis=3)
att = att.eval().reshape((im_height, im_width))
sess.close()
plt.imshow(att, interpolation='gaussian')
plt.show()